In [1]:
import pandas as pd
import numpy as np

In [17]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'pGtcK4Cfe0bYhkzj57xpd52SoW35u9AminIgvd2fgQxN',
    'service_id': 'iam-ServiceId-8270013a-9afb-4c42-9b56-f62850a9c0f4',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_9fa254825cf748af8c99d9684609cb33_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initialized for you.
# The following variable contains the path to your file on your IBM Cloud Object Storage.
movies_path = cos.url('movielens_movies.dat', 'sparkscalapythonplayground-donotdelete-pr-pmmafeibz2hwiv')
ratings_path = cos.url('movielens_ratings.dat', 'sparkscalapythonplayground-donotdelete-pr-pmmafeibz2hwiv')
users_path = cos.url('movielens_users.dat', 'sparkscalapythonplayground-donotdelete-pr-pmmafeibz2hwiv')


# Read movies file

In [11]:
movies_df = spark.read.options(delimiter=':').csv(movies_path)

In [12]:
movies_df.take(1)

[Row(_c0='1', _c1=None, _c2='Toy Story (1995)', _c3=None, _c4="Animation|Children's|Comedy")]

In [15]:
movies_df = movies_df.drop('_c1', '_c3')

In [16]:
movies_df.take(1)

[Row(_c0='1', _c2='Toy Story (1995)', _c4="Animation|Children's|Comedy")]

In [35]:
movies_df = movies_df.withColumn('MovieId', movies_df._c0).withColumn('Title', movies_df._c2).withColumn('Genre', movies_df._c4)
movies_df.take(1)

[Row(_c0='1', _c2='Toy Story (1995)', _c4="Animation|Children's|Comedy", MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]

In [36]:
movies_df = movies_df.drop('_c0','_c2','_c4')
movies_df.take(1)

[Row(MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]

In [19]:
ratings_df = spark.read.options(delimiter=':').csv(ratings_path)
ratings_df.take(1)

[Row(_c0='1', _c1=None, _c2='1193', _c3=None, _c4='5', _c5=None, _c6='978300760')]

In [20]:
ratings_df = ratings_df.drop('_c1','_c3','_c5')
ratings_df.take(1)

[Row(_c0='1', _c2='1193', _c4='5', _c6='978300760')]

In [25]:
ratings_df = ratings_df.withColumn('UserID', ratings_df._c0).withColumn('MoviesId', ratings_df._c2).withColumn('Rating', ratings_df._c4).withColumn('Timestamp', ratings_df._c6)
ratings_df.take(1)

[Row(_c0='1', _c2='1193', _c4='5', _c6='978300760', UserID_='1', MoviesId='1193', Rating='5', Timestamp='978300760', UserID='1')]

In [27]:
ratings_df = ratings_df.drop('_c0','_c2','_c4','_c6','UserID_')
ratings_df.take(1)

[Row(MoviesId='1193', Rating='5', Timestamp='978300760', UserID='1')]

In [30]:
users_df = spark.read.options(delimiter=':').csv(users_path)
users_df.take(1)

[Row(_c0='1', _c1=None, _c2='F', _c3=None, _c4='1', _c5=None, _c6='10', _c7=None, _c8='48067')]

In [31]:
users_df = users_df.drop('_c1','_c3','_c5','_c7')
users_df.take(1)

[Row(_c0='1', _c2='F', _c4='1', _c6='10', _c8='48067')]

In [32]:
users_df = users_df.withColumn('UserId', users_df._c0).withColumn('Gender', users_df._c2).withColumn('Age', users_df._c4).withColumn('Occupation', users_df._c6).withColumn('ZipCode', users_df._c8)
users_df.take(1)

[Row(_c0='1', _c2='F', _c4='1', _c6='10', _c8='48067', UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]

In [34]:
users_df = users_df.drop('_c0','_c2','_c4','_c6','_c8')
users_df.take(1)

[Row(UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]

In [37]:
age_dict = {"1":  "Under 18",'18':  "18-24",'25':  "25-34",'35':  "35-44",'45':  "45-49",'50':  "50-55", '56':  "56+"}
age_dict

{'1': 'Under 18',
 '18': '18-24',
 '25': '25-34',
 '35': '35-44',
 '45': '45-49',
 '50': '50-55',
 '56': '56+'}

In [38]:
occ_dict = {'0':  "other or not specified", '1':  "academic/educator", '2':  "artist",'3':  "clerical/admin",'4':  "college/grad student",\
            '5':  "customer service",'6':  "doctor/health care",'7':  "executive/managerial",'8':  "farmer",'9':  "homemaker",\
            '10':  "K-12 student",'11':  "lawyer",'12':  "programmer",'13':  "retired",'14':  "sales/marketing",'15':  "scientist",\
            '16':  "self-employed",'17':  "technician/engineer",'18':  "tradesman/craftsman",'19':  "unemployed",'20':  "writer"}
occ_dict

{'0': 'other or not specified',
 '1': 'academic/educator',
 '10': 'K-12 student',
 '11': 'lawyer',
 '12': 'programmer',
 '13': 'retired',
 '14': 'sales/marketing',
 '15': 'scientist',
 '16': 'self-employed',
 '17': 'technician/engineer',
 '18': 'tradesman/craftsman',
 '19': 'unemployed',
 '2': 'artist',
 '20': 'writer',
 '3': 'clerical/admin',
 '4': 'college/grad student',
 '5': 'customer service',
 '6': 'doctor/health care',
 '7': 'executive/managerial',
 '8': 'farmer',
 '9': 'homemaker'}

In [39]:
print(movies_df.take(1))
print(ratings_df.take(2))
print(users_df.take(1))


[Row(MovieId='1', Title='Toy Story (1995)', Genre="Animation|Children's|Comedy")]
[Row(MoviesId='1193', Rating='5', Timestamp='978300760', UserID='1'), Row(MoviesId='661', Rating='3', Timestamp='978302109', UserID='1')]
[Row(UserId='1', Gender='F', Age='1', Occupation='10', ZipCode='48067')]
